In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [54]:
downsample_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/downsample_data.csv')

1. Generic Extra tree regression Model FSO_Att

In [55]:
# choose only top 14 features which produce the optimal model
X = downsample_data.drop(columns=['FSO_Att','RFL_Att','Frequency','WindDirection',
                                  'RainIntensityMax','RainIntensityMin','RainIntensity',
                                   'WindSpeed','WindSpeedMin','SYNOPCode','TemperatureDifference',
                                   'WindSpeedMax', 'ParticulateMax','AbsoluteHumidityMin' ])
y_fso = downsample_data['FSO_Att']

In [56]:
X

,AbsoluteHumidity,AbsoluteHumidityMax,Distance,Particulate,ParticulateMin,RelativeHumidity,Temperature,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin
0,20.244567,20.819340,2115.667536,0.000000,0.000000,97.366554,23.201133,25.021254,21.063413,17,141.152237,154.806868,129.044488
1,20.164626,20.833908,2119.138170,0.000000,0.000000,98.006488,23.017296,25.182210,20.854708,17,2936.215800,2985.174438,2898.281083
2,17.515085,18.017632,2962.180194,0.000000,0.000000,82.951956,23.470992,24.086457,22.270371,14,26762.246420,27056.350834,24252.617383
3,17.796149,18.858701,2964.409031,0.000000,0.000000,84.561544,23.413134,24.634976,23.073526,14,9383.749728,9504.425544,8736.247346
4,17.943657,17.995399,2963.216445,0.000000,0.000000,87.517871,22.956074,23.263533,22.161422,15,578.014621,598.699391,577.802421
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,9.383492,10.103008,2012.267998,14.072659,12.726364,67.166083,16.422097,17.954418,14.862038,15,53018.676810,53337.116489,51869.200609
1333,19.386257,19.538264,2961.452049,19.982826,18.756192,82.001620,25.469319,25.702272,23.133961,15,19962.163690,20269.867813,18049.145610
1334,20.457950,21.183070,2958.056188,0.000000,0.000000,98.362391,23.206548,24.304511,23.110257,18,34560.594040,37136.535922,33031.971421
1335,5.717047,6.069735,4822.782771,117.289839,115.306987,81.012923,5.571695,5.623157,5.365653,0,25393.310090,27868.232161,22970.299437


In [57]:
# Split the data
X_train_fso, X_test_fso, y_train_fso, y_test_fso = train_test_split(X, y_fso, test_size=0.25, random_state=42)
# Initial ExtraTreesRegressor with OOB Score
et_fso = ExtraTreesRegressor(random_state=42, oob_score=True, bootstrap=True)

In [60]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=et_fso, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_fso, y_train_fso)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(cv=5,
             estimator=ExtraTreesRegressor(bootstrap=True, oob_score=True,
                                           random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, None],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [61]:
best_model = grid_search.best_estimator_

# Predictions
y_pred_fso = best_model.predict(X_test_fso)

best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Calculate RMSE
rmse = round(np.sqrt(mean_squared_error(y_test_fso, y_pred_fso)),2)
print(f"Root Mean Squared Error (RMSE) on Test Data: {rmse}")

# Calculate R² score
r2 = round(r2_score(y_test_fso, y_pred_fso),2)
print(f"R-squared (R²) on Test Data: {r2}")


Best Parameters: {'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Root Mean Squared Error (RMSE) on Test Data: 1.55
R-squared (R²) on Test Data: 0.93
